In [29]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import tifffile as tf
from skimage import io
from PIL import Image
import sys
sys.path.append(r'C:\Users\jinho.kim\Github\lamf_ophys_analysis_dev')
from utils import zstack_reg

def get_zstack_reg(zstack, num_slices, ref_ind, proj='mean'):
    plane_reg_imgs = np.zeros((num_slices, zstack.shape[1], zstack.shape[2]))
    shift_all_planes = []
    for plane_ind in range(num_slices):
        temp_imgs = zstack[plane_ind::num_slices]
        if proj == 'mean':
            reg_img, shift_all = zstack_reg.average_reg_plane(temp_imgs)
        elif proj == 'max':
            reg_img, shift_all = zstack_reg.maxproj_reg_plane(temp_imgs)
        else:
            raise ValueError('proj should be either mean or max')
        plane_reg_imgs[plane_ind] = reg_img
        shift_all_planes.append(shift_all)
    between_plane_reg_imgs, shift_all_between_planes = zstack_reg.reg_between_planes(np.array(plane_reg_imgs), ref_ind=ref_ind)
    return between_plane_reg_imgs, shift_all_planes, shift_all_between_planes

def get_zstack_reg_using_shifts(zstack, num_slices, shift_all_planes, shift_all_between_planes,
                                proj='mean'):
    plane_reg_imgs = np.zeros((num_slices, zstack.shape[1], zstack.shape[2]))
    for plane_ind in range(num_slices):
        temp_imgs = zstack[plane_ind::num_slices]
        if proj == 'mean':
            reg_img = zstack_reg.maxproj_reg_plane_using_shift_info(temp_imgs, shift_all_planes[plane_ind])
        elif proj == 'max':
            reg_img = zstack_reg.maxproj_reg_plane_using_shift_info(temp_imgs, shift_all_planes[plane_ind])
        else:
            raise ValueError('proj should be either mean or max')
        plane_reg_imgs[plane_ind] = reg_img
    between_plane_reg_imgs = zstack_reg.reg_between_planes_using_shift_info(np.array(plane_reg_imgs), shift_all_between_planes)
    return between_plane_reg_imgs

def save_zstack_reg(zstack_fn, save_dir, ref_ind=60):
    si_metadata = zstack_reg.read_si_stack_metadata(zstack_fn)
    zstack = tf.imread(zstack_fn)
    assert zstack.shape[0] == si_metadata['num_slices'] * si_metadata['num_volumes']
    num_slices = si_metadata['num_slices']
    assert len(zstack.shape) == 3

    zstack_registered, shift_all_planes, shift_all_between_planes = get_zstack_reg(zstack, num_slices, ref_ind,
                                                                                   proj='mean')
    tf.imsave(save_dir / f'{zstack_fn.stem}_reg_mean.tif', zstack_registered)
    shift_info = {'shift_all_planes': shift_all_planes, 'shift_all_between_planes': shift_all_between_planes}
    np.save(save_dir / f'{zstack_fn.stem}_reg_shift_info.npy', shift_info)

    zstack_ch_reg = get_zstack_reg_using_shifts(zstack, num_slices, shift_all_planes, shift_all_between_planes,
                                                proj='max')
    tf.imsave(save_dir / f'{zstack_fn.stem}_reg_max.tif', zstack_ch_reg)

In [5]:
si_metadata = zstack_reg.read_si_stack_metadata(cortical_zstack_fn)

In [8]:
si_metadata['num_volumes']

100

In [9]:
zstack = tf.imread(cortical_zstack_fn)
print(zstack.shape)

(40000, 512, 512)


In [16]:
from importlib import reload
reload(zstack_reg)

<module 'utils.zstack_reg' from 'C:\\Users\\jinho.kim\\Github\\lamf_ophys_analysis_dev\\utils\\zstack_reg.py'>

In [ ]:
# About 70 min
cortical_zstack_fn = Path(r'\\allen\programs\mindscope\production\omfish\prod0\specimen_1291107612\ophys_session_1325739462\1325739462_cortical_z_stack0.tiff')
save_dir = Path(r'\\allen\programs\mindscope\workgroups\learning\coreg\multi-slice pilot\693124')
save_zstack_reg(cortical_zstack_fn, save_dir)

In [17]:
shift_info = np.load(save_dir / f'{cortical_zstack_fn.stem}_reg_shift_info.npy', allow_pickle=True).item()

In [25]:
shift_all_planes = shift_info['shift_all_planes']
shift_all_between_planes = shift_info['shift_all_between_planes']

In [23]:
num_slices = len(shift_info['shift_all_between_planes'])

In [30]:
zstack_ch_reg = get_zstack_reg_using_shifts(zstack, num_slices, shift_all_planes, shift_all_between_planes,
                                                proj='max')

In [31]:
tf.imsave(save_dir / f'{cortical_zstack_fn.stem}_reg_max.tif', zstack_ch_reg)

C:\Users\jinho.kim\AppData\Local\Temp\ipykernel_43548\3355411629.py:1: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tf.imsave(save_dir / f'{cortical_zstack_fn.stem}_reg_max.tif', zstack_ch_reg)
